In [ ]:
# python file



import datetime 
from datetime import date, timedelta
import os
import pycurl
import requests

import os 
from lxml import etree
import pandas as pd 
import datetime
import requests
from bs4 import BeautifulSoup




def create_history_html_folder():
    folder_name = datetime.datetime.now().strftime('history_folder_%Y-%m-%d')   
    os.chdir('/home/jupyter-ed/projects/history')
    # define the name of the directory to be created
    os.mkdir(folder_name)
    return(folder_name)
    

def get_daily_inventory():
  url = 'https://www.reginfo.gov/public/do/PRAXML?type=inventory'
  r = requests.get(url)
  xml_file_name = datetime.datetime.now().strftime('%Y-%m-%d_inventory.xml')
  
  with open(xml_file_name, 'wb') as f:
    f.write(r.content)
    
  return(xml_file_name)

    

def inventory_to_omb_cntrl_number_list(file,dol_xpath_strings):
    tree = etree.parse(file)
    root = tree.getroot()
    omb_cntrl_number_list = []

    for string in xpath_strings:
      req = root.xpath(string)
      for request in req:
        omb_cntrl_number_list.append(request.xpath('./OMBControlNumber//text()')[0])
     
    return(omb_cntrl_number_list)  



def history_table_link_generator(omb_control_number):
    reg_info_slug = "https://www.reginfo.gov/public/do/PRAOMBHistory?ombControlNumber="
    reg_info_url = reg_info_slug + omb_control_number
    return(reg_info_url)
  
    
def history_link_list_creator(omb_cntrl_number_list):
    omb_history_url_list = []
    
    for number in omb_cntrl_number_list:
        reg_info_url = history_table_link_generator(number)
        omb_history_url_list.append(reg_info_url)    
    
    return(omb_history_url_list)
    


def get_history_html_file(praurl,omb_control_number):
    filename = omb_control_number +'.html'    
    file = open(filename,'wb')
    crl = pycurl.Curl()
    crl.setopt(crl.URL, praurl)
    crl.setopt(crl.WRITEDATA, file)
    crl.perform()
    crl.close()

def hmtl_list_generator(omb_cntrl_number_list,omb_history_url_list):
    list_range = range(len(omb_cntrl_number_list))
    for number in list_range:
        omb_cntrl_instance = omb_cntrl_number_list[number]
        praurl = omb_history_url_list[number]
        get_history_html_file(praurl,omb_cntrl_instance)
        

def get_first_row_history(file):
    with open(file) as f:
        content = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
        content = [x.strip() for x in content] 
        omb_data = {"icr_ref_number" : content[778],
                    "request_type" : content[785].replace('&nbsp;',""),
                    "received_by_omb"  : content[788].replace('&nbsp;',""),
                    "conclusion_date"  : content[794].replace('&nbsp;',""),
                    "conclusion_action"  : content[798],
                    "omb_control_number": file[0:9]}
        return(omb_data)


dol_agency_codes = [
1220,
1225,
1205,
1219,
1218,
1250,
1245,
1240,
1293,
1290,
1235,
1210]

xpath_strings = []

xpath_string_prefix = '//InformationCollectionRequest[.//AgencyCode/text()[. = "'
xpath_string_suffix = '"]]'

for number in dol_agency_codes:
    new_string = xpath_string_prefix+str(number)+xpath_string_suffix
    xpath_strings.append(new_string)  




        

todays_folder_name = create_history_html_folder()

os.chdir(todays_folder_name)


print('getting inventory')  
xml_file_name = get_daily_inventory()     

omb_cntrl_number_list = inventory_to_omb_cntrl_number_list(xml_file_name,xpath_strings)

omb_history_url_list = history_link_list_creator(omb_cntrl_number_list)

print('getting html')

hmtl_list_generator(omb_cntrl_number_list,omb_history_url_list)

rows = []

print('get first row')
for file in os.listdir():
    if file.endswith(".html"):
        data=get_first_row_history(file)
        rows.append(data)


row_range = range(len(rows))



for number  in row_range:
    try:
        rows[number]['icr_ref_number'] = rows[number]['icr_ref_number'].split('>')[1].split('<')[0]
    except:
        rows[number]['icr_ref_number'] = rows[number]['icr_ref_number']       


for number  in row_range:
    try:
        rows[number]['conclusion_action'] = rows[number]['conclusion_action'].split('>')[1].split('<')[0]
    except:
        rows[number]['conclusion_action'] = rows[number]['conclusion_action']   

print('history table')
history_table = pd.DataFrame(rows)


history_table['received_by_omb'] = pd.to_datetime(history_table['received_by_omb'])

history_table['conclusion_date'] = pd.to_datetime(history_table['conclusion_date'])

datetime.datetime.now().strftime('%Y-%m-%d_inventory.xml')

dt = date.today() - timedelta(5)

dt = dt.strftime('%Y-%m-%d')

filter_date = dt

mask = (history_table['received_by_omb'] > filter_date) | (history_table['conclusion_date'] > filter_date) 

week_table = history_table.loc[mask]
print('writeout')
week_table.to_csv('week_table_'+date.today().strftime('%Y-%m-%d')+'.csv')





In [2]:

import datetime 
from datetime import date, timedelta
import os
import pycurl
import requests

import os 
from lxml import etree
import pandas as pd 
import datetime
import requests
from bs4 import BeautifulSoup


In [3]:
os.getcwd()

'/home/jupyter-ed/projects/pra/scripts/EDA'

In [4]:
history = '/home/jupyter-ed/projects/pra/data/history'

In [9]:
os.chdir(history)

In [10]:
os.listdir()

['history_folder_2020-06-29',
 'history_folder_2020-07-01',
 'history_folder_2020-06-25',
 'history_folder_2020-06-24',
 'history_folder_2020-06-27',
 'history_folder_2020-06-28']

In [11]:
folder ='history_folder_2020-07-01'
os.chdir(folder)
files=os.listdir()
file1 =files[0]

In [27]:
files

['1218-0180.html',
 '1219-0039.html',
 '1220-0186.html',
 '1205-0030.html',
 '1210-0064.html',
 '1210-0090.html',
 '1205-0432.html',
 '1240-0007.html',
 '1220-0017.html',
 '1205-0540.html',
 '1210-0092.html',
 '1210-0122.html',
 '1240-0049.html',
 '1210-0116.html',
 '1293-0014.html',
 '1218-0061.html',
 '1218-0101.html',
 '1205-0541.html',
 '1205-0177.html',
 '1205-0530.html',
 '1235-0005.html',
 '1219-0120.html',
 '1220-0189.html',
 '1210-0091.html',
 '1220-0104.html',
 '1235-0003.html',
 '1210-0076.html',
 '1205-0527.html',
 '1205-0371.html',
 '1205-0353.html',
 '1240-0021.html',
 '1218-0266.html',
 '1205-0132.html',
 '1220-0109.html',
 '1218-0230.html',
 '1235-0013.html',
 '1240-0003.html',
 '1240-0040.html',
 '3046-0046.html',
 '1219-0096.html',
 '1210-0144.html',
 '1245-0004.html',
 '1210-0131.html',
 '1219-0140.html',
 '1218-0267.html',
 '1290-0022.html',
 '1210-0084.html',
 '1219-0011.html',
 '1218-0242.html',
 '1220-0042.html',
 '1240-0024.html',
 '1219-0024.html',
 '1210-0040.

In [22]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


def get_first_row_history(file):
    with open(file) as f:
        content = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
        content = [x.strip() for x in content] 
        omb_data = {"icr_ref_number" : remove_html_tags(content[778]),
                    "request_type" : content[785].replace('&nbsp;',""),
                    "received_by_omb"  : content[788].replace('&nbsp;',""),
                    "conclusion_date"  : content[794].replace('&nbsp;',""),
                    "conclusion_action"  : remove_html_tags(content[798]),
                    "omb_control_number": remove_html_tags(file[0:9])}
        return(omb_data)

In [25]:
history_info =[]

In [28]:
for file in files:
    history_info.append(get_first_row_history(file))
    

In [29]:
df_history = pd.DataFrame(history_info)

In [30]:
df_history

,icr_ref_number,request_type,received_by_omb,conclusion_date,conclusion_action,omb_control_number
0,201809-1218-019,Revision of a currently approved collection,05/14/2019,08/29/2019,Approved without change,1218-0180
1,201904-1219-006,Extension without change of a currently approv...,07/29/2019,10/07/2019,Approved without change,1219-0039
2,201812-1220-002,Reinstatement without change of a previously a...,05/29/2019,07/03/2019,Approved without change,1220-0186
3,202004-1205-002,Extension without change of a currently approv...,04/28/2020,05/29/2020,Approved without change,1205-0030
4,201908-1210-003,Extension without change of a currently approv...,11/22/2019,01/06/2020,Approved without change,1210-0064
...,...,...,...,...,...,...
464,201906-1220-002,Extension without change of a currently approv...,10/02/2019,12/13/2019,Approved with change,1220-0133
465,201707-1219-001,Extension without change of a currently approv...,04/20/2018,07/09/2018,Approved without change,1219-0066
466,201701-1235-002,Extension without change of a currently approv...,06/30/2017,08/30/2017,Approved without change,1235-0002
467,201809-1218-001,New collection (Request for a new OMB Control ...,11/09/2018,02/11/2019,Approved without change,1218-0270


In [31]:
df_history['conclusion_date'] = pd.to_datetime(df_history['conclusion_date'])
df_history['received_by_omb'] = pd.to_datetime(df_history['received_by_omb'])

In [34]:
sorted_df_history = df_history.sort_values(by=['received_by_omb'],ascending=False)


In [35]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(sorted_df_history)

,icr_ref_number,request_type,received_by_omb,conclusion_date,conclusion_action,omb_control_number
244,201905-1225-001,Generic IC,2020-06-30,NaT,,1225-0086
424,201708-1220-004,Generic IC,2020-06-29,NaT,,1220-0141
363,202006-1220-004,No material or nonsubstantive change to a curr...,2020-06-29,NaT,,1220-0163
36,202002-1240-002,Revision of a currently approved collection,2020-06-25,NaT,,1240-0003
390,202004-1205-005,Extension without change of a currently approv...,2020-06-24,NaT,,1205-0528
346,202001-1219-001,Extension without change of a currently approv...,2020-06-23,NaT,,1219-0089
445,202002-1219-005,Extension without change of a currently approv...,2020-06-23,NaT,,1219-0003
109,202005-1290-001,No material or nonsubstantive change to a curr...,2020-06-22,2020-06-24,Approved without change,1290-0029
197,202005-1218-001,Extension without change of a currently approv...,2020-06-18,NaT,,1218-0143
182,202006-1220-001,No material or nonsubstantive change to a curr...,2020-06-17,2020-06-19,Approved without change,1220-0196


{'icr_ref_number': '201809-1218-019',
 'request_type': 'Revision of a currently approved collection',
 'received_by_omb': '05/14/2019',
 'conclusion_date': '08/29/2019',
 'conclusion_action': 'Approved without change',
 'omb_control_number': '1218-0180'}

In [17]:

remove_html_tags(fileinfo['conclusion_action'])


'Approved without change'

In [ ]:
import datetime 
from datetime import date, timedelta
import os
import pycurl
import requests

import os 
from lxml import etree
import pandas as pd 
import datetime
import requests
from bs4 import BeautifulSoup




def get_first_row_history(file):
    with open(file) as f:
        content = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
        content = [x.strip() for x in content] 
        omb_data = {"icr_ref_number" : content[778],
                    "request_type" : content[785].replace('&nbsp;',""),
                    "received_by_omb"  : content[788].replace('&nbsp;',""),
                    "conclusion_date"  : content[794].replace('&nbsp;',""),
                    "conclusion_action"  : content[798],
                    "omb_control_number": file[0:9]}
        return(omb_data)
    
    
 rows = []

print('get first row')
for file in os.listdir():
    if file.endswith(".html"):
        data=get_first_row_history(file)
        rows.append(data)


row_range = range(len(rows))



for number  in row_range:
    try:
        rows[number]['icr_ref_number'] = rows[number]['icr_ref_number'].split('>')[1].split('<')[0]
    except:
        rows[number]['icr_ref_number'] = rows[number]['icr_ref_number']       


for number  in row_range:
    try:
        rows[number]['conclusion_action'] = rows[number]['conclusion_action'].split('>')[1].split('<')[0]
    except:
        rows[number]['conclusion_action'] = rows[number]['conclusion_action']   

print('history table')
history_table = pd.DataFrame(rows)


history_table['received_by_omb'] = pd.to_datetime(history_table['received_by_omb'])

history_table['conclusion_date'] = pd.to_datetime(history_table['conclusion_date'])

datetime.datetime.now().strftime('%Y-%m-%d_inventory.xml')

dt = date.today() - timedelta(5)

dt = dt.strftime('%Y-%m-%d')

filter_date = dt

mask = (history_table['received_by_omb'] > filter_date) | (history_table['conclusion_date'] > filter_date) 

week_table = history_table.loc[mask]
print('writeout')
week_table.to_csv('week_table_'+date.today().strftime('%Y-%m-%d')+'.csv')
   
    
    